Install PyTorch and Other Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import cv2
import numpy as np


ModuleNotFoundError: No module named 'torch'

Image enhance technique 

In [ ]:
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Image Enhancement Function
def enhance_image(image):
    # Convert to OpenCV format
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    
    # Apply histogram equalization
    equalized = cv2.equalizeHist(image)
    
    # Convert to PyTorch tensor and apply preprocessing
    equalized = preprocess(equalized)
    
    return equalized


UNet Model

In [ ]:
class UNetModel(nn.Module):
    def __init__(self):
        super(UNetModel, self).__init__()
        # Define U-Net architecture
        ...

    def forward(self, x):
        # Forward pass through U-Net
        ...
        return segmentation_mask


ResNet Model

In [ ]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        # Load pre-trained ResNet
        self.resnet = torchvision.models.resnet50(pretrained=True)
        # Modify the last layer for binary classification
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        # Forward pass through ResNet
        output = self.resnet(x)
        return output


In [ ]:
# Combined Model
def predict(tile_image):
    # Load models
    unet_model = UNetModel()
    resnet_model = ResNetModel()

    # Enhance and resize the input tile image
    enhanced_image = enhance_image(tile_image)

    # Perform semantic segmentation using U-Net
    segmentation_mask = unet_model(enhanced_image)
    road_pixels, building_pixels = count_pixels(segmentation_mask)

    # Perform image-level classification using ResNet
    resnet_output = resnet_model(enhanced_image)
    resnet_output = torch.softmax(resnet_output, dim=1)
    _, pred_class = torch.max(resnet_output, 1)

    # Combine outputs from U-Net and ResNet
    if road_pixels > building_pixels and pred_class == 0:  # 0 for "road-dominant"
        return "Roads are more prominent"
    elif building_pixels > road_pixels and pred_class == 1:  # 1 for "building-dominant"
        return "Buildings are more prominent"
    else:
        return "Inconclusive"

# Helper function to count pixels in the segmentation mask
def count_pixels(mask):
    road_pixels = torch.sum(mask == 0)  # Assuming 0 is the road class
    building_pixels = torch.sum(mask == 1)  # Assuming 1 is the building class
    return road_pixels, building_pixels

# Example usage
tile_image = # Load an OpenStreetMap tile image
result = predict(tile_image)
print(result)